<h1> <center> Table of Contents </center> </h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#1.-Required-Libraries"> Required Libraries</a></li>
        <li><a href="#2.-Importing-the-Data"> Importing the Data</a></li>
        <li><a href="#3.-Creating-Corpus"> Creating-Corpus</a> </li>
      </li>
    </ol>
</div>

In [1]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

# 1. Required Libraries 

## Dependencies


In [2]:
#!pip install python-Levenshtein
#!pip install "fuzzywuzzy==0.18.0"
#!pip install rapidfuzz
#!pip install spacy
#!pip install gensim
#!pip install rank_bm25
#!python -m spacy download en_core_web_lg
#!pip install fast-autocomplete

## Imports 

In [3]:
import pandas as pd
from pandas.core.common import flatten
import numpy as np

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import rapidfuzz
from rapidfuzz import process, utils

import timeit
import time
import warnings
warnings.filterwarnings('ignore')

#For each character
from pynput.keyboard import Listener,Key

#importing the fuzzy logic from different python file
from ipynb.fs.full.fuzzy_module import init

/Users/rasheshkothari/opt/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## 2. Importing the Data

In [4]:
#importing the file
# Might have to change the file names
dfdashboards = pd.read_csv('microstrategy_and_dashboard-Copy1.csv')
dfmetrics = pd.read_csv('tableau_metric-Copy1.csv')
dfuserquey = pd.read_csv('sample_user_searchs-Copy1.csv')

In [5]:
def preprocess_datafiles(dashboard , metric):

    #converting column values to lowercase
    dashboard = dashboard.apply(lambda x: x.astype(str).str.lower())
    metric = metric.apply(lambda x: x.astype(str).str.lower())
    
    #converting global usage to numeric
    dashboard.Global_usage = pd.to_numeric(dashboard.Global_usage)
    
    #splitting caption and descriptors columns 
    dashboard = dashboard[['dashboards','caption','Global_usage']]
    dashboard['metric_names'] = dashboard.caption.str.split('|')
    metric['descriptors'] = metric.descriptors.str.split('|')
    
    #converting metric names and descriptors in each row values to multiple rows

    temp1 = dashboard.set_index(['dashboards'])['metric_names'].apply(pd.Series).stack().reset_index().drop('level_1', axis=1).rename(columns={0:'caption'})
    temp2 = metric.set_index(['metric_name'])['descriptors'].apply(pd.Series).stack().reset_index().drop('level_1', axis=1).rename(columns={0:'descriptor'})
    
    #merging metrics and descriptors

    master = temp1.merge(temp2, how = 'inner', left_on ='caption', right_on = 'metric_name')[['dashboards','metric_name','descriptor']]
    master = master.rename(columns={"dashboards":"dashboard_names", "metric_name": "metric_names", "descriptor": "descriptor_names"})

    return master




In [6]:
#"This is the final dataframe we would be using."
master = preprocess_datafiles(dfdashboards,dfmetrics)

## 3. Creating Corpus

In [7]:
dashboard_names_list = list(master.dashboard_names.unique())
metric_names_list = list(master.metric_names.unique())
descriptor_names_list = list(master.descriptor_names.unique())
mastercorpus = dashboard_names_list + metric_names_list + descriptor_names_list

mastercorpus = list(filter(None,mastercorpus))

## Key Listener

1. Currently, key listener will work for any <b>Alphabets</b> and <b>Numeric</b> key pressed.  
2. For any <b> Spaces </b> and <b> backspaces </b>, it will take into account. But it will call the fuzzylogic moodule. Which is inefficient and need to work on that.  
3. <b> Enter </b> and <b> Escape </b> will stop the Listener.  
4. <b> Other Keys </b> will work same as Spaces


In [14]:
buffer = ''
def on_press(key):
#     f = open(filename, 'a')  # Open the file
    global buffer 
    if hasattr(key, 'char'):  # Write the character pressed if available
        x = (key.char)
    elif key == Key.space:  # If space was pressed, write a space
        x = (' ')
        
    elif ((key == Key.alt)     | 
          (key == Key.cmd)     | 
          (key == Key.ctrl)    | 
          (key == Key.cmd)     | 
          (key == Key.delete)  | 
          (key == Key.down)    |  
          (key == Key.tab)     | 
          (key == Key.shift)   | 
          (key == Key.shift_r) | 
          (key == Key.shift_l) | 
          (key == Key.up)      |
          (key == Key.down)    |
          (key == Key.left)    |
          (key == Key.right)) :    
           
        x = ''
    elif ((key == Key.enter) | ( key == Key.esc)):
        # Stop listener
        return False
    elif key == Key.backspace:
        if buffer != "":
            x = buffer[0:-1]
            buffer = ""
        else:
            x = ""

    buffer += str(x).replace("'", "")
    
    if (hasattr(key, 'char') | (key == Key.backspace)) :
        
        init(str(buffer),mastercorpus, master,dashboard_names_list,metric_names_list)




In [15]:
with Listener(on_press=on_press) as listener:  # Setup the listener
    listener.join()  # Join the thread to the main thread

User Search : m


,dashboard_names,metric_names,descriptor_names
0,brand control,"[select main brand family, select main vendor]","[m i a brand family, m special brand family, d p m co, m s walker inc, m special brewing co]"
1,distribution gains and losses cube vs. last month,"[store name, store street address]","[m & r pkg manchester, m & r pkg s windsor, m & r pkg avon, antonio m llc., 10 n m 149]"
2,distribution gains and losses cube vs. last year,"[store name, store street address]","[m & r pkg manchester, m & r pkg s windsor, m & r pkg avon, antonio m llc., 10 n m 149]"
3,retail cube to brand quality level,"[store name, store street address]","[m & r pkg manchester, m & r pkg s windsor, m & r pkg avon, antonio m llc., 10 n m 149]"
4,retail cube to brand level,"[store name, store street address]","[m & r pkg manchester, m & r pkg s windsor, m & r pkg avon, antonio m llc., 10 n m 149]"
5,retail cube with reorder to rollup,"[store name, store street address]","[m & r pkg manchester, m & r pkg s windsor, m & r pkg avon, antonio m llc., 10 n m 149]"
6,retail month cube,"[store name, store street address]","[m & r pkg manchester, m & r pkg s windsor, m & r pkg avon, antonio m llc., 10 n m 149]"
7,sales manager dashboard,"[store name, store street address]","[m & r pkg manchester, m & r pkg s windsor, m & r pkg avon, antonio m llc., 10 n m 149]"
8,sales team opportunitshis year,"[store name, store street address]","[m & r pkg manchester, m & r pkg s windsor, m & r pkg avon, antonio m llc., 10 n m 149]"
9,store detail to this year vs. last year,"[store name, store street address]","[m & r pkg manchester, m & r pkg s windsor, m & r pkg avon, antonio m llc., 10 n m 149]"


User Search : ma


,dashboard_names,metric_names,descriptor_names
0,depletions month cube with inventory and shipmentments,"[distributor, sales state, month]","[m.s. walker ma (norwood), horizon beverage ma (norton), may, ma]"
1,depletionsletions cube,"[distributor, sales state]","[m.s. walker ma (norwood), horizon beverage ma (norton), ma]"
2,market index analysis,"[comparison geography, primary geography]","[boston, ma liquor, total wine &amp; more ma rma liquor, boston, ma liquor, total wine &amp; more ma rma liquor]"
3,market index analysis visual,"[comparison geography, primary geography]","[boston, ma liquor, total wine &amp; more ma rma liquor, boston, ma liquor, total wine &amp; more ma rma liquor]"
4,price promotion count,"[comparison geography, primary geography]","[boston, ma liquor, total wine &amp; more ma rma liquor, boston, ma liquor, total wine &amp; more ma rma liquor]"
5,the humbler,"[comparison geography, primary geography]","[boston, ma liquor, total wine &amp; more ma rma liquor, boston, ma liquor, total wine &amp; more ma rma liquor]"


User Search : mac


...

,dashboard_names,metric_names,descriptor_names
0,ad analysis,"[beer style levels, price per case, segment, beer style level value]","[macro style value, micro style valu, macro style value, micro style value, beer style, 11]"
1,aggregate sales per pt vs. cwd,"[beer style levels, price per case, segment, beer style level value]","[macro style value, micro style valu, macro style value, micro style value, beer style, 11]"
2,brands on ad,"[beer style levels, price per case, segment, beer style level value]","[macro style value, micro style valu, macro style value, micro style value, beer style, 11]"
3,brewery comparisons,"[beer style levels, price per case, segment, beer style level value]","[macro style value, micro style valu, macro style value, micro style value, beer style, 11]"
4,category and segment analysis,"[beer style levels, price per case, segment, beer style level value]","[macro style value, micro style valu, macro style value, micro style value, beer style, 11]"
5,company,"[beer style levels, price per case, segment, beer style level value]","[macro style value, micro style valu, macro style value, micro style value, beer style, 11]"
6,company,"[beer style levels, price per case, segment, beer style level value]","[macro style value, micro style valu, macro style value, micro style value, beer style, 11]"
7,company brands over time,"[beer style levels, price per case, segment, beer style level value]","[macro style value, micro style valu, macro style value, micro style value, beer style, 11]"
8,r geography,"[beer style levels, price per case, segment, beer style level value]","[macro style value, micro style valu, macro style value, micro style value, beer style, 11]"
9,r geography over time,"[beer style levels, price per case, segment, beer style level value]","[macro style value, micro style valu, macro style value, micro style value, beer style, 11]"


User Search : macro style 11 ct


,dashboard_names,metric_names,descriptor_names
0,depletions month cube with inventory and shipmentments,[sales state],[ct]
1,depletionsletions,[sales state],[ct]
2,depletionsletions cube,[sales state],[ct]
3,retail cube to brand quality level,"[store street address, sales state]","[ct, 1241 east main st, 11]"
4,retail cube to brand level,"[store street address, sales state]","[ct, 1241 east main st, 11]"
5,retail cube with reorder to rollup,"[store street address, sales state, territory]","[ct, 1241 east main st, 11, ct]"
6,retail month cube,"[store street address, sales state, territory]","[ct, 1241 east main st, 11, ct]"
7,sales manager dashboard,"[store street address, sales state]","[ct, 1241 east main st, 11]"
8,sales team opportunitshis year,"[store street address, sales state]","[ct, 1241 east main st, 11]"
9,sales team tracker,"[store street address, sales state]","[ct, 1241 east main st, 11]"


User Search : macro style 11 ct 7


,dashboard_names,metric_names,descriptor_names
0,ad analysis,"[beer style levels, package size group, beer style level value, price per case, segment]","[macro style value, macro style value, 11 ct 7 oz bottle, 12 ct 7 oz bottle, 3 ct 11 oz bottle, 1 ct 12.7 oz bottle, 1 ct 7 oz bottle, beer style, 7, 11]"
1,aggregate sales per pt vs. cwd,"[beer style levels, package size group, beer style level value, price per case, segment]","[macro style value, macro style value, 11 ct 7 oz bottle, 12 ct 7 oz bottle, 3 ct 11 oz bottle, 1 ct 12.7 oz bottle, 1 ct 7 oz bottle, beer style, 7, 11]"
2,brands on ad,"[beer style levels, package size group, beer style level value, price per case, segment]","[macro style value, macro style value, 11 ct 7 oz bottle, 12 ct 7 oz bottle, 3 ct 11 oz bottle, 1 ct 12.7 oz bottle, 1 ct 7 oz bottle, beer style, 7, 11]"
3,brewery comparisons,"[beer style levels, package size group, beer style level value, price per case, segment]","[macro style value, macro style value, 11 ct 7 oz bottle, 12 ct 7 oz bottle, 3 ct 11 oz bottle, 1 ct 12.7 oz bottle, 1 ct 7 oz bottle, beer style, 7, 11]"
4,category and segment analysis,"[beer style levels, package size group, beer style level value, price per case, segment]","[macro style value, macro style value, 11 ct 7 oz bottle, 12 ct 7 oz bottle, 3 ct 11 oz bottle, 1 ct 12.7 oz bottle, 1 ct 7 oz bottle, beer style, 7, 11]"
5,company,"[beer style levels, package size group, beer style level value, price per case, segment]","[macro style value, macro style value, 11 ct 7 oz bottle, 12 ct 7 oz bottle, 3 ct 11 oz bottle, 1 ct 12.7 oz bottle, 1 ct 7 oz bottle, beer style, 7, 11]"
6,company,"[beer style levels, package size group, beer style level value, price per case, segment]","[macro style value, macro style value, 11 ct 7 oz bottle, 12 ct 7 oz bottle, 3 ct 11 oz bottle, 1 ct 12.7 oz bottle, 1 ct 7 oz bottle, beer style, 7, 11]"
7,company brands over time,"[beer style levels, package size group, beer style level value, price per case, segment]","[macro style value, macro style value, 11 ct 7 oz bottle, 12 ct 7 oz bottle, 3 ct 11 oz bottle, 1 ct 12.7 oz bottle, 1 ct 7 oz bottle, beer style, 7, 11]"
8,r geography,"[beer style levels, package size group, beer style level value, price per case, segment]","[macro style value, macro style value, 11 ct 7 oz bottle, 12 ct 7 oz bottle, 3 ct 11 oz bottle, 1 ct 12.7 oz bottle, 1 ct 7 oz bottle, beer style, 7, 11]"
9,r geography over time,"[beer style levels, package size group, beer style level value, price per case, segment]","[macro style value, macro style value, 11 ct 7 oz bottle, 12 ct 7 oz bottle, 3 ct 11 oz bottle, 1 ct 12.7 oz bottle, 1 ct 7 oz bottle, beer style, 7, 11]"


# 

11  ct 